In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn import preprocessing
import warnings

warnings.filterwarnings("ignore")

import datetime

In [2]:
import matplotlib.pyplot as plt

%matplotlib inline

In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [4]:
df = pd.read_csv('train.csv', index_col='ID_code')

In [51]:
prediction = pd.read_csv('test.csv', index_col='ID_code')

In [9]:
df.shape

(200000, 201)

In [8]:
df.head()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [8]:
df.isnull().sum().sum()

0

In [6]:
y = df['target'].values
del df['target']
X = df.values

print(y.shape, X.shape)
notras, tras = np.bincount([int(i) for i in y ])
print('%s notras, and %s tras' % (notras, tras))

(200000,) (200000, 200)
179902 notras, and 20098 tras


# Holdout

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2)

In [22]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [23]:
params = {
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric':'auc',
          'learning_rate': 0.2,
          'num_leaves': 35,  # we should let it be smaller than 2^(max_depth)
          #'max_depth': 7,  # -1 means no limit
          'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
          'subsample': 0.3,  # Subsample ratio of the training instance.
          'subsample_freq': 3,  # frequence of subsample, <=0 means no enable
          'colsample_bytree': 0.8,  # Subsample ratio of columns when constructing each tree.
          'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
          'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
          'verbose': 0,
          'boost_from_average': False,
          #'scale_pos_weight':1, # because training data is extremely unbalanced 
          'is_unbalance': True,
          'nthread': -1
         }

In [24]:
print('Start training...')
# train
gbm = lgb.train(params,
lgb_train,
num_boost_round=5000,
valid_sets=lgb_eval,
early_stopping_rounds=10)

Start training...
[1]	valid_0's auc: 0.652878
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.682301
[3]	valid_0's auc: 0.702824
[4]	valid_0's auc: 0.723509
[5]	valid_0's auc: 0.738907
[6]	valid_0's auc: 0.749782
[7]	valid_0's auc: 0.758928
[8]	valid_0's auc: 0.766433
[9]	valid_0's auc: 0.771291
[10]	valid_0's auc: 0.777987
[11]	valid_0's auc: 0.784012
[12]	valid_0's auc: 0.787867
[13]	valid_0's auc: 0.79117
[14]	valid_0's auc: 0.794919
[15]	valid_0's auc: 0.799765
[16]	valid_0's auc: 0.80431
[17]	valid_0's auc: 0.807016
[18]	valid_0's auc: 0.809615
[19]	valid_0's auc: 0.812555
[20]	valid_0's auc: 0.81625
[21]	valid_0's auc: 0.818687
[22]	valid_0's auc: 0.820888
[23]	valid_0's auc: 0.822895
[24]	valid_0's auc: 0.824161
[25]	valid_0's auc: 0.826766
[26]	valid_0's auc: 0.829324
[27]	valid_0's auc: 0.830801
[28]	valid_0's auc: 0.832733
[29]	valid_0's auc: 0.834164
[30]	valid_0's auc: 0.835525
[31]	valid_0's auc: 0.836594
[32]	valid_0's auc: 0.837319
[33]

In [38]:
gbm.best_score['valid_0']['auc']

0.8649360003310618

# K-fold CV

In [9]:
params = {
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric':'auc',
          'learning_rate': 0.1,
          #'max_depth': 7,  # -1 means no limit
          'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
          'verbose': 0,
          'boost_from_average': False,
          #'scale_pos_weight':1, # because training data is extremely unbalanced 
          'is_unbalance': True,
          'nthread': -1
         }

In [16]:
gridParams = {
    'num_leaves': [10,15,25,35],
    'colsample_bytree' : [0.6, 0.8],
    'subsample' : [0.3,0.5,0.7],
    'subsample_freq': [0,1,3,5],
    #'n_estimators': [50000],
    #'early_stopping_rounds': [20]
    }

In [17]:
mdl = lgb.LGBMClassifier(boosting_type= 'gbdt',
          objective = 'binary',
          silent = True,
          learning_rate = 0.1,
          min_child_samples = 100,
          boost_from_average=False,
          is_unbalance = True)

In [18]:
grid = GridSearchCV(mdl, gridParams,
                    verbose=0,
                    cv=4,
                    n_jobs=-1)

In [19]:
grid.fit(X, y)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
D:\workingsoftware\anaconda\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...onda\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
D:\workingsoftware\anaconda\lib\ru

In [163]:
print(grid.best_params_)
print(grid.best_score_)

{'colsample_bytree': 0.8, 'num_leaves': 35, 'subsample': 0.3, 'subsample_freq': 3}
0.81714


# K-fold

In [71]:
params = {
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric':'auc',
          'learning_rate': 0.01,
          'num_leaves': 25,  # we should let it be smaller than 2^(max_depth)
          #'max_depth': 7,  # -1 means no limit
          'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
          'subsample': 0.3,  # Subsample ratio of the training instance.
          'subsample_freq': 3,  # frequence of subsample, <=0 means no enable
          'colsample_bytree': 0.8,  # Subsample ratio of columns when constructing each tree.
          'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
          'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
          #'verbose': 0,
          'boost_from_average': False,
          #'scale_pos_weight':1, # because training data is extremely unbalanced 
          #'is_unbalance': True,
          'nthread': -1
         }

In [72]:
NFOLDS = 10
kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=1)
kf = kfold.split(X, y)


In [73]:
score = 0
y_pred = np.zeros(len(prediction))
for i, (train, test) in enumerate(kf):
    X_train = X[train,:]
    X_test = X[test,:]
    y_train = y[train]
    y_test = y[test]
    dtrain = lgb.Dataset(X_train, y_train)
    dvalid = lgb.Dataset(X_test, y_test, reference=dtrain)
    bst = lgb.train(params, 
                    dtrain, 
                    num_boost_round=50000, 
                    valid_sets=dvalid, 
                    early_stopping_rounds=20)
    print(bst.best_score['valid_0']['auc'])
    score = score + bst.best_score['valid_0']['auc']
    y_pred = y_pred + bst.predict(prediction)

score = score/NFOLDS
print('cv score for valid is: ', score)


[1]	valid_0's auc: 0.657354
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.670284
[3]	valid_0's auc: 0.677842
[4]	valid_0's auc: 0.694237
[5]	valid_0's auc: 0.69703
[6]	valid_0's auc: 0.699415
[7]	valid_0's auc: 0.706497
[8]	valid_0's auc: 0.709098
[9]	valid_0's auc: 0.711985
[10]	valid_0's auc: 0.717781
[11]	valid_0's auc: 0.720352
[12]	valid_0's auc: 0.725609
[13]	valid_0's auc: 0.730118
[14]	valid_0's auc: 0.730418
[15]	valid_0's auc: 0.731694
[16]	valid_0's auc: 0.734172
[17]	valid_0's auc: 0.737191
[18]	valid_0's auc: 0.738284
[19]	valid_0's auc: 0.739255
[20]	valid_0's auc: 0.739389
[21]	valid_0's auc: 0.740979
[22]	valid_0's auc: 0.741721
[23]	valid_0's auc: 0.742919
[24]	valid_0's auc: 0.742426
[25]	valid_0's auc: 0.743275
[26]	valid_0's auc: 0.744796
[27]	valid_0's auc: 0.745015
[28]	valid_0's auc: 0.745938
[29]	valid_0's auc: 0.746834
[30]	valid_0's auc: 0.747868
[31]	valid_0's auc: 0.748473
[32]	valid_0's auc: 0.74955
[33]	valid_0's auc: 0

[277]	valid_0's auc: 0.813661
[278]	valid_0's auc: 0.813991
[279]	valid_0's auc: 0.814458
[280]	valid_0's auc: 0.81457
[281]	valid_0's auc: 0.814647
[282]	valid_0's auc: 0.814715
[283]	valid_0's auc: 0.814831
[284]	valid_0's auc: 0.814855
[285]	valid_0's auc: 0.814872
[286]	valid_0's auc: 0.815119
[287]	valid_0's auc: 0.815333
[288]	valid_0's auc: 0.815477
[289]	valid_0's auc: 0.815763
[290]	valid_0's auc: 0.816085
[291]	valid_0's auc: 0.816442
[292]	valid_0's auc: 0.816565
[293]	valid_0's auc: 0.816797
[294]	valid_0's auc: 0.81694
[295]	valid_0's auc: 0.817088
[296]	valid_0's auc: 0.817347
[297]	valid_0's auc: 0.817407
[298]	valid_0's auc: 0.817628
[299]	valid_0's auc: 0.817649
[300]	valid_0's auc: 0.81776
[301]	valid_0's auc: 0.817957
[302]	valid_0's auc: 0.818227
[303]	valid_0's auc: 0.818452
[304]	valid_0's auc: 0.818666
[305]	valid_0's auc: 0.818796
[306]	valid_0's auc: 0.818967
[307]	valid_0's auc: 0.81899
[308]	valid_0's auc: 0.819101
[309]	valid_0's auc: 0.819317
[310]	valid_0'

[552]	valid_0's auc: 0.851722
[553]	valid_0's auc: 0.851759
[554]	valid_0's auc: 0.851834
[555]	valid_0's auc: 0.851917
[556]	valid_0's auc: 0.851952
[557]	valid_0's auc: 0.852072
[558]	valid_0's auc: 0.852142
[559]	valid_0's auc: 0.852186
[560]	valid_0's auc: 0.852195
[561]	valid_0's auc: 0.852323
[562]	valid_0's auc: 0.852385
[563]	valid_0's auc: 0.852393
[564]	valid_0's auc: 0.852519
[565]	valid_0's auc: 0.852616
[566]	valid_0's auc: 0.852714
[567]	valid_0's auc: 0.852838
[568]	valid_0's auc: 0.852916
[569]	valid_0's auc: 0.852934
[570]	valid_0's auc: 0.853077
[571]	valid_0's auc: 0.853167
[572]	valid_0's auc: 0.853306
[573]	valid_0's auc: 0.853457
[574]	valid_0's auc: 0.853544
[575]	valid_0's auc: 0.853629
[576]	valid_0's auc: 0.853694
[577]	valid_0's auc: 0.853792
[578]	valid_0's auc: 0.853943
[579]	valid_0's auc: 0.854054
[580]	valid_0's auc: 0.854128
[581]	valid_0's auc: 0.854221
[582]	valid_0's auc: 0.854339
[583]	valid_0's auc: 0.85449
[584]	valid_0's auc: 0.854662
[585]	valid

[828]	valid_0's auc: 0.870626
[829]	valid_0's auc: 0.870664
[830]	valid_0's auc: 0.870705
[831]	valid_0's auc: 0.870751
[832]	valid_0's auc: 0.870752
[833]	valid_0's auc: 0.870792
[834]	valid_0's auc: 0.870851
[835]	valid_0's auc: 0.870913
[836]	valid_0's auc: 0.870973
[837]	valid_0's auc: 0.87102
[838]	valid_0's auc: 0.871049
[839]	valid_0's auc: 0.871082
[840]	valid_0's auc: 0.871082
[841]	valid_0's auc: 0.871111
[842]	valid_0's auc: 0.871147
[843]	valid_0's auc: 0.871203
[844]	valid_0's auc: 0.871261
[845]	valid_0's auc: 0.871286
[846]	valid_0's auc: 0.871344
[847]	valid_0's auc: 0.871378
[848]	valid_0's auc: 0.871432
[849]	valid_0's auc: 0.871508
[850]	valid_0's auc: 0.87156
[851]	valid_0's auc: 0.871602
[852]	valid_0's auc: 0.871642
[853]	valid_0's auc: 0.871712
[854]	valid_0's auc: 0.87181
[855]	valid_0's auc: 0.871921
[856]	valid_0's auc: 0.871984
[857]	valid_0's auc: 0.872046
[858]	valid_0's auc: 0.872143
[859]	valid_0's auc: 0.872169
[860]	valid_0's auc: 0.872176
[861]	valid_0

[1100]	valid_0's auc: 0.88012
[1101]	valid_0's auc: 0.880131
[1102]	valid_0's auc: 0.880187
[1103]	valid_0's auc: 0.880223
[1104]	valid_0's auc: 0.880254
[1105]	valid_0's auc: 0.880245
[1106]	valid_0's auc: 0.880226
[1107]	valid_0's auc: 0.880239
[1108]	valid_0's auc: 0.880282
[1109]	valid_0's auc: 0.880284
[1110]	valid_0's auc: 0.88034
[1111]	valid_0's auc: 0.88036
[1112]	valid_0's auc: 0.880388
[1113]	valid_0's auc: 0.880416
[1114]	valid_0's auc: 0.880433
[1115]	valid_0's auc: 0.88043
[1116]	valid_0's auc: 0.880418
[1117]	valid_0's auc: 0.880454
[1118]	valid_0's auc: 0.880457
[1119]	valid_0's auc: 0.880486
[1120]	valid_0's auc: 0.880524
[1121]	valid_0's auc: 0.880569
[1122]	valid_0's auc: 0.880582
[1123]	valid_0's auc: 0.880632
[1124]	valid_0's auc: 0.880672
[1125]	valid_0's auc: 0.880703
[1126]	valid_0's auc: 0.880741
[1127]	valid_0's auc: 0.880771
[1128]	valid_0's auc: 0.880788
[1129]	valid_0's auc: 0.8808
[1130]	valid_0's auc: 0.880789
[1131]	valid_0's auc: 0.880765
[1132]	valid_0

...

[1671]	valid_0's auc: 0.889393
[1672]	valid_0's auc: 0.889402
[1673]	valid_0's auc: 0.889417
[1674]	valid_0's auc: 0.889424
[1675]	valid_0's auc: 0.889463
[1676]	valid_0's auc: 0.889497
[1677]	valid_0's auc: 0.889514
[1678]	valid_0's auc: 0.889505
[1679]	valid_0's auc: 0.889525
[1680]	valid_0's auc: 0.889522
[1681]	valid_0's auc: 0.889535
[1682]	valid_0's auc: 0.889546
[1683]	valid_0's auc: 0.889545
[1684]	valid_0's auc: 0.889574
[1685]	valid_0's auc: 0.889603
[1686]	valid_0's auc: 0.889607
[1687]	valid_0's auc: 0.889614
[1688]	valid_0's auc: 0.889624
[1689]	valid_0's auc: 0.889635
[1690]	valid_0's auc: 0.889658
[1691]	valid_0's auc: 0.88967
[1692]	valid_0's auc: 0.889664
[1693]	valid_0's auc: 0.889652
[1694]	valid_0's auc: 0.889668
[1695]	valid_0's auc: 0.889682
[1696]	valid_0's auc: 0.889682
[1697]	valid_0's auc: 0.889692
[1698]	valid_0's auc: 0.889716
[1699]	valid_0's auc: 0.889728
[1700]	valid_0's auc: 0.889711
[1701]	valid_0's auc: 0.889722
[1702]	valid_0's auc: 0.889737
[1703]	va

[1937]	valid_0's auc: 0.892041
[1938]	valid_0's auc: 0.892053
[1939]	valid_0's auc: 0.892057
[1940]	valid_0's auc: 0.892064
[1941]	valid_0's auc: 0.892079
[1942]	valid_0's auc: 0.892103
[1943]	valid_0's auc: 0.892111
[1944]	valid_0's auc: 0.892123
[1945]	valid_0's auc: 0.89211
[1946]	valid_0's auc: 0.892128
[1947]	valid_0's auc: 0.892143
[1948]	valid_0's auc: 0.892142
[1949]	valid_0's auc: 0.892141
[1950]	valid_0's auc: 0.892158
[1951]	valid_0's auc: 0.89217
[1952]	valid_0's auc: 0.892174
[1953]	valid_0's auc: 0.892171
[1954]	valid_0's auc: 0.892177
[1955]	valid_0's auc: 0.892157
[1956]	valid_0's auc: 0.892157
[1957]	valid_0's auc: 0.892176
[1958]	valid_0's auc: 0.892218
[1959]	valid_0's auc: 0.892235
[1960]	valid_0's auc: 0.892249
[1961]	valid_0's auc: 0.89225
[1962]	valid_0's auc: 0.892257
[1963]	valid_0's auc: 0.892259
[1964]	valid_0's auc: 0.892261
[1965]	valid_0's auc: 0.892284
[1966]	valid_0's auc: 0.892313
[1967]	valid_0's auc: 0.892311
[1968]	valid_0's auc: 0.892332
[1969]	vali

[2205]	valid_0's auc: 0.893741
[2206]	valid_0's auc: 0.893751
[2207]	valid_0's auc: 0.893746
[2208]	valid_0's auc: 0.893738
[2209]	valid_0's auc: 0.893732
[2210]	valid_0's auc: 0.893742
[2211]	valid_0's auc: 0.893743
[2212]	valid_0's auc: 0.893762
[2213]	valid_0's auc: 0.893775
[2214]	valid_0's auc: 0.893795
[2215]	valid_0's auc: 0.893802
[2216]	valid_0's auc: 0.893798
[2217]	valid_0's auc: 0.893798
[2218]	valid_0's auc: 0.893813
[2219]	valid_0's auc: 0.893823
[2220]	valid_0's auc: 0.893838
[2221]	valid_0's auc: 0.893852
[2222]	valid_0's auc: 0.893858
[2223]	valid_0's auc: 0.893849
[2224]	valid_0's auc: 0.893862
[2225]	valid_0's auc: 0.893843
[2226]	valid_0's auc: 0.893827
[2227]	valid_0's auc: 0.893839
[2228]	valid_0's auc: 0.893855
[2229]	valid_0's auc: 0.893863
[2230]	valid_0's auc: 0.893875
[2231]	valid_0's auc: 0.893869
[2232]	valid_0's auc: 0.893867
[2233]	valid_0's auc: 0.893864
[2234]	valid_0's auc: 0.893869
[2235]	valid_0's auc: 0.893866
[2236]	valid_0's auc: 0.893877
[2237]	v

[2471]	valid_0's auc: 0.895232
[2472]	valid_0's auc: 0.895231
[2473]	valid_0's auc: 0.895249
[2474]	valid_0's auc: 0.895252
[2475]	valid_0's auc: 0.895243
[2476]	valid_0's auc: 0.895234
[2477]	valid_0's auc: 0.895241
[2478]	valid_0's auc: 0.895262
[2479]	valid_0's auc: 0.895255
[2480]	valid_0's auc: 0.895255
[2481]	valid_0's auc: 0.895253
[2482]	valid_0's auc: 0.895268
[2483]	valid_0's auc: 0.895301
[2484]	valid_0's auc: 0.895336
[2485]	valid_0's auc: 0.895346
[2486]	valid_0's auc: 0.895339
[2487]	valid_0's auc: 0.895339
[2488]	valid_0's auc: 0.895354
[2489]	valid_0's auc: 0.895374
[2490]	valid_0's auc: 0.89538
[2491]	valid_0's auc: 0.895367
[2492]	valid_0's auc: 0.895375
[2493]	valid_0's auc: 0.895377
[2494]	valid_0's auc: 0.89538
[2495]	valid_0's auc: 0.895386
[2496]	valid_0's auc: 0.895382
[2497]	valid_0's auc: 0.895398
[2498]	valid_0's auc: 0.895398
[2499]	valid_0's auc: 0.895405
[2500]	valid_0's auc: 0.895433
[2501]	valid_0's auc: 0.895441
[2502]	valid_0's auc: 0.895466
[2503]	val

[2739]	valid_0's auc: 0.896319
[2740]	valid_0's auc: 0.896342
[2741]	valid_0's auc: 0.896355
[2742]	valid_0's auc: 0.896375
[2743]	valid_0's auc: 0.896354
[2744]	valid_0's auc: 0.896344
[2745]	valid_0's auc: 0.896348
[2746]	valid_0's auc: 0.896339
[2747]	valid_0's auc: 0.89633
[2748]	valid_0's auc: 0.896335
[2749]	valid_0's auc: 0.896329
[2750]	valid_0's auc: 0.89633
[2751]	valid_0's auc: 0.89634
[2752]	valid_0's auc: 0.896347
[2753]	valid_0's auc: 0.896361
[2754]	valid_0's auc: 0.896365
[2755]	valid_0's auc: 0.896379
[2756]	valid_0's auc: 0.896379
[2757]	valid_0's auc: 0.896405
[2758]	valid_0's auc: 0.896414
[2759]	valid_0's auc: 0.896417
[2760]	valid_0's auc: 0.896418
[2761]	valid_0's auc: 0.896402
[2762]	valid_0's auc: 0.896401
[2763]	valid_0's auc: 0.896395
[2764]	valid_0's auc: 0.896397
[2765]	valid_0's auc: 0.896412
[2766]	valid_0's auc: 0.896415
[2767]	valid_0's auc: 0.896414
[2768]	valid_0's auc: 0.896425
[2769]	valid_0's auc: 0.896426
[2770]	valid_0's auc: 0.896448
[2771]	vali

# Predict

In [189]:
print('Start predicting...')
# predict
y_prob = gbm.predict(test, num_iteration=gbm.best_iteration)
# eval
print(y_prob)

Start predicting...
[0.47028108 0.62129473 0.65957063 ... 0.02272149 0.29321187 0.33208792]


In [77]:
f = open("result.csv", "w+")

f.write('ID_code,target\r')
for i in range(len(y_prob)):
    f.write(str(prediction.index[i]))
    f.write(',')
    f.write(str(y_prob[i]))
    f.write('\r')
    
f.close()